In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import zscore

# Sample dataset
data = {
    'ClaimAmount': [1000, 1200, None, 5000, 300, None, 1500],
    'ClaimType': ['Medical', 'Dental', 'Medical', 'Medical', 'Dental', 'Dental', 'Medical'],
    'ServiceDate': ['01/02/21', '2021-03-05', '04-15-2021', '03/10/2021', '15-04-2021', None, '2021/06/25']
}

df = pd.DataFrame(data)

# -----------------------
# 1. Impute Nulls by Group
# -----------------------
def impute_claim_amount(df):
    def impute(group):
        median = group['ClaimAmount'].median()
        return group['ClaimAmount'].fillna(median)
    
    return df.groupby('ClaimType', group_keys=False).apply(lambda g: g.assign(ClaimAmount=impute(g)))

df['ClaimAmount'] = impute_claim_amount(df)

# -----------------------
# 2. Flag Outliers using Z-Score
# -----------------------
df['ZScore'] = zscore(df['ClaimAmount'])
df['IsOutlier'] = df['ZScore'].abs() > 2

# -----------------------
# 3. Convert Date to ISO Format
# -----------------------
def convert_to_iso(date):
    if pd.isna(date):
        return pd.NaT
    for fmt in ('%m/%d/%y', '%Y-%m-%d', '%m-%d-%Y', '%d-%m-%Y', '%Y/%m/%d'):
        try:
            return pd.to_datetime(date, format=fmt)
        except:
            continue
    return pd.NaT

df['FormattedDate'] = df['ServiceDate'].apply(convert_to_iso)

# Drop helper column
df.drop(columns='ZScore', inplace=True)

print(df)
